In [6]:
catagories = {'C01':'Bacterial Infections and Mycoses',
'C02':'Virus Diseases',
'C03':'Parasitic Diseases',
'C04':'Neoplasms',
'MusCuloskeletal Diseases':'C05',
'Digestive System Diseases':'C06',
'Stomatognathic Diseases':'C07',
'Respiratory Tract Diseases':'C08',
'Otorhinolaryngologic Diseases':'C09',
'Nervous System Diseases':'C10',
'Eye Diseases':'C11',
'Urologic and Male Genital Diseases':'C12',
'Female Genital Diseases and Pregnancy Complications':'C13',
'CardiovasCular Diseases':'C14',
'Hemic and Lymphatic Diseases':'C15',
'Neonatal Diseases and Abnormalities':'C16',
'Skin and Connective Tissue Diseases':'C17',
'Nutritional and Metabolic Diseases':'C18',
'Endocrine Diseases':'C19',
'Immunologic Diseases':'C20',
'Disorders of Environmental Origin':'C21',
'Animal Diseases':'C22',
'Pathological Conditions, Signs and Symptoms':'C23'}

In [17]:
from __future__ import print_function

import logging
import numpy as np
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from sklearn.utils.extmath import density
from sklearn import metrics

In [18]:
dataset_folder = "./ohsumed-first-20000-docs"

In [21]:
training_folder = dataset_folder + '/training'
test_folder = dataset_folder + '/test'

In [22]:
test_data = load_files(container_path=test_folder, categories=None, load_content=True, shuffle=True, encoding='utf-8', random_state=17)
training_data = load_files(container_path=training_folder, categories=None, load_content=True, shuffle=True, encoding='utf-8', random_state=17)

In [23]:
categories = training_data.target_names

In [24]:
print("categories values: ",categories)

categories values:  ['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23']


In [25]:
y_train,y_test = training_data.target , test_data.target

array([22, 22,  8, ..., 22,  0,  5])

In [ ]:
t